In [37]:
import polars as pl
import pandas as pd
from PyPDF2 import PdfReader
import re
import os
from dbfread import DBF

In [38]:
ROOT_DIR = "C:\\Users\\nicho\\Documents\\research\\ga-tax-assessment\\data\\paulding\\parcels\\"
FORMAT_FILE = r"C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2012 DIGEST VENDOR FILES\Paulding_GA_Record_Layout.pdf"
file_path = r'C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2012 DIGEST VENDOR FILES\2012 PAULDING_GA_DIGEST.txt'
out_file = r'C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2012 DIGEST VENDOR FILES\converted_2012.txt'

IGNORE_WORDS = ["","Description", "Banker", "Paulding County", "characters", "1", "2", "3"]

In [41]:
dirs = [d for d in os.listdir(ROOT_DIR)]
txt_method_dirs = []
dbf_method_dirs = []
dfs = []

for d in dirs:
    if re.findall('2011|2012|2013', d): #'2011|2012|2013'
        txt_method_dirs.append(d)
    elif re.findall('2', d):
        dbf_method_dirs.append(d)

# for d in txt_method_dirs:
#     print(ROOT_DIR + d)
#     dfs.append(convert_txt_to_df(ROOT_DIR + d))


for d in dbf_method_dirs:
    print(d)
    #dfs.append(convert_dbf_to_df(d)) # drop uneeded

# # make sure to add file metadata first

# df = pl.concat(dfs)


2014 Digest Vendor File
2015 Digest Vendor File
2016 Digest Vendor File
2017 Digest Vendor File
2018 DIGEST VENDOR FILES
2019 DIGEST VENDOR FILES
2023 PAULDING_GA_DIGEST VENDOR FILES
Paulding_GA.Public Export_2021
Paulding_GA_2020_publicexport
Paulding_GA_2022_Public Export


In [34]:
def get_indices(text):
    pattern = '(-\d+)'
    indices = [re.findall(pattern, s) for s in text]
    indices = [int(l[0].replace("-", "")) for l in indices if len(l) > 0]
    indices = [n for n in indices if n != None]
    return indices

def get_columns(text):
    pattern = r'[A-Za-z ]+'
    columns = [re.findall(pattern, s) for s in text]
    columns = [col[0].replace("  ", " ").strip() for col in columns if len(col) > 0]

    columns = [s for s in columns if (s != None and s not in IGNORE_WORDS)]
    unique = dict()
    for i, col in enumerate(columns):
        if col not in unique:
            unique[col] = 1
        else:
            columns[i] = col + " " + str(unique[col])
            unique[col] += 1
    return columns

def insert_commas_at_indices(file_path, out_file, indices):
    # Read the file content
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Sort indices in reverse order to avoid index shift when inserting
    for i, line in enumerate(lines):
        for index in indices:
            line = line[:index] + ',' + line[index:]
        if i < 2:
            print(line)
            print(line.replace('\n', '').strip().strip(',') + '\n' )
        lines[i] = line.replace('\n', ' ').strip().lstrip(',') + '\n'  # Update the line in the 'lines' list

    # Write the modified content back to the file
    with open(out_file, 'w') as file:
        file.writelines(lines)

def convert_txt_to_df(directory):
    filename = [f for f in os.listdir(directory) if '.txt' in f][0]
    layout_name = [f for f in os.listdir(directory) if '.pdf' in f][0]
    out_filename = 'converted_' + filename.replace(" ", "_")
    out_path = directory + "\\" + out_filename
    file_path = directory + "\\" + filename
    layout_path = directory + "\\" + layout_name

    print(f"reading {file_path}")
    print(f"layout at {layout_path}")
    print(f"out at {out_path}")
    reader = PdfReader(layout_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    text = text.split('\n')

    indices = get_indices(text)

    if "2011" in directory: # special case :(
        indices += [1141]

    indices = sorted(indices, reverse=True)[1:]
    columns = get_columns(text)
    insert_commas_at_indices(file_path, out_path, indices)

    df = pl.read_csv(out_path, infer_schema_length=0)
    df.columns = columns

    return df

In [ ]:
def convert_dbf_to_df(directory, ROOT_DIR):
    

In [17]:
dbf = DBF(r"C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2015 Digest Vendor File\PE_TAXDETAIL.dbf")

In [20]:
df = pl.from_pandas(pd.DataFrame(iter(dbf)))

In [22]:
df.sample(5)

REALKEY,PERSKEY,TAXTYPE,TT_DESC,ASMTPCT,COVEXMPT,HSEXEMPT,MILLAGE,ESTTAX,ACCKEY
i64,i64,i64,str,f64,i64,i64,f64,f64,i64
17746,0,4,"""County Bond""",0.4,0,0,0.00222,89.04,0
45827,0,3,"""County M&O""",0.4,0,2000,0.007267,540.96,0
38731,0,6,"""State""",0.4,0,2000,0.0001,3.65,0
68319,0,5,"""County Fire""",0.4,0,0,0.0031,21.7,0
30917,0,4,"""County Bond""",0.4,0,0,0.00222,169.18,0


In [23]:
dbf = DBF(r"C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2015 Digest Vendor File\PE_OWNER.dbf")
df = pl.from_pandas(pd.DataFrame(iter(dbf)))
df.sample(5)

OWNKEY,LASTNAME,FIRSTNAME,MIDDLE,ADDRESS1,ADDRESS2,ADDRESS3,CITY,STATE,ZIP,HOMEPHONE,WORKPHONE,FAXNUMBER,FEI,SSN,SSN1,SST,TAXRETURN,ACCTSTATUS,MULTIOWNER,BIRTHDATE1,BIRTHDATE2,EXISTS,NO_RELEASE,CREATEDATE,EMAIL,CELLPHONE,COUNTRY,SITEADDID
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,bool,bool,date,str,str,str,str
104193,"""RATCLIFF SONJA…","""""","""""","""""","""PO BOX 587""","""""","""DALLAS""","""GA""","""30132""","""""","""""","""""","""""","""""","""""","""""","""""",false,null,null,null,false,false,null,"""""","""""","""""",""""""
47241,"""VILLANI EDWARD…","""""","""""","""""","""22 CREEKSIDE V…","""""","""HIRAM""","""GA""","""30141""","""""","""""","""""","""""","""""","""""","""""","""""",false,null,null,null,false,false,null,"""""","""""","""""",""""""
46253,"""TROPEANO DEBRA…","""""","""""","""""","""293 HARRIS OAK…","""""","""DALLAS""","""GA""","""30157""","""""","""""","""""","""""","""""","""""","""""","""""",false,null,null,null,false,false,null,"""""","""""","""""",""""""
174906,"""WILLIAMS DANA""","""""","""""","""""","""41 LADY SAVANN…","""""","""DALLAS""","""GA""","""30157""","""""","""""","""""","""""","""""","""""","""""","""""",true,null,null,null,false,false,2015-02-10,"""""","""""","""""",""""""
147910,"""WIX MICHAEL WA…","""""","""""","""""","""117 FAITHFUL D…","""""","""TEMPLE""","""GA""","""30179""","""""","""""","""""","""""","""""","""""","""""","""""",false,null,null,null,false,false,null,"""""","""""","""""",""""""


In [24]:
dbf = DBF(r"C:\Users\nicho\Documents\research\ga-tax-assessment\data\paulding\parcels\2015 Digest Vendor File\PE_REALPROP.dbf")
df = pl.from_pandas(pd.DataFrame(iter(dbf)))
df.sample(5)

REALKEY,OWNKEY,HOUSE_NO,EXTENSION,STDIRECT,STTYPE,STREET_NAM,UNIT,LANDLOT,LANDDIST,LANDGMD,ZONINGCODE,COMMENT1,RETURN_VAL,ASSESS_RSN,PARCEL_NO,LEGAL_DESC,VAL_CHG,PREV_VAL,CURR_VAL,VALCHGDATE,LAND_TYPE,TAXDISTRIC,HOMEEXEMPT,CUV_RENEW,ORIGHOMVAL,CURRHOMVAL,REVIEWDATE,DATENOW,APPRAISER,PCY,CCY,HCY,OVRIDEVAL,INFLUENCE1,INFLUENCE2,INFLUENCE3,…,STATEHSVAL,FLCY,FL08VAL,FLBASEVAL,FL08ACRES,FLAPPNUM,FL08PERAC,MAVORIG,MAVCURR,MAVAG,MAVPREF,MAVPREV,MAVOVR,FMVRES,FMVCOM,FMVACC,MAVRES,MAVCOM,MAVACC,SALE_VAL,SALELAND1,SALELAND2,SALEIMP1,SALEIMP2,SALEIMPCS1,SALEIMPCS2,ADFACTOR,RETURNMAIL,ORIGHSVAL2,CURRHSVAL2,COVHS,COVPARENT,COVREALKEY,SITEADDID,UNITTYPE,COMMENTFLG,A299C
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,bool,i64,i64,date,i64,str,str,bool,i64,i64,date,date,str,str,str,str,i64,f64,f64,f64,…,i64,str,i64,i64,f64,str,f64,i64,i64,i64,i64,i64,bool,i64,i64,i64,i64,i64,i64,bool,i64,i64,i64,i64,str,str,f64,bool,i64,i64,bool,str,i64,str,str,bool,bool
2167,65967,208,"""""","""""","""""","""HILLSIDE DR""","""""","""40""","""2""","""3""","""""",null,0,"""""","""107320100000""","""REAL PROPERTY""",false,137230,135800,null,1,"""01""","""S1""",false,0,0,2013-04-11,2013-12-03,"""""","""""","""""","""""",0,0.0,0.0,0.0,…,0,"""""",0,0,0.0,"""""",0.0,0,0,0,0,0,false,93200,0,2600,0,0,0,false,0,0,0,0,"""""","""""",0.0,false,0,0,false,"""""",0,"""""","""""",false,false
25587,74796,295,"""""","""""","""""","""MICHAEL CT""","""""","""177""","""2""","""3""","""""",null,0,"""52""","""112320070000""","""REAL PROPERTY""",false,77070,90440,null,1,"""01""","""S1""",false,0,0,2013-11-07,2015-03-30,"""svenenga""","""""","""""","""""",0,0.0,0.0,0.0,…,0,"""""",0,0,0.0,"""""",0.0,0,0,0,0,0,false,69800,0,640,0,0,0,false,0,0,0,0,"""""","""""",0.0,false,0,0,false,"""""",0,"""""","""""",false,false
45074,138927,16,"""""","""""","""""","""COLBY CT""","""""","""998""","""19""","""3""","""""",null,0,"""52""","""198320200000""","""REAL PROPERTY""",false,47230,79520,null,1,"""01""","""S1""",false,0,0,null,2014-07-17,"""tcooper""","""""","""""","""""",0,0.0,0.0,0.0,…,0,"""""",0,0,0.0,"""""",0.0,0,0,0,0,0,false,61400,0,3120,0,0,0,false,0,0,0,0,"""""","""""",0.0,false,0,0,false,"""""",0,"""""","""""",false,false
54601,112800,144,"""""","""""","""""","""COLONIAL WAY""","""""","""324""","""2""","""3""","""""",null,0,"""52""","""142220030000""","""REAL PROPERTY""",false,190500,237100,null,1,"""01""","""L7""",false,0,0,2013-09-25,2015-03-10,"""svenenga""","""""","""""","""""",0,0.0,0.0,0.0,…,237100,"""""",0,0,0.0,"""""",0.0,0,0,0,0,0,false,193400,0,5700,0,0,0,false,0,0,0,0,"""""","""""",0.0,false,0,0,false,"""""",0,"""""","""""",false,false
51868,135920,62,"""""","""""","""""","""GROVE PL""","""""","""776""","""19""","""3""","""""",null,0,"""""","""193110060000""","""REAL PROPERTY""",false,65760,80470,null,1,"""01""","""S1""",false,0,0,2013-05-28,2013-12-03,"""""","""""","""""","""""",0,0.0,0.0,0.0,…,0,"""""",0,0,0.0,"""""",0.0,0,0,0,0,0,false,64500,0,970,0,0,0,false,0,0,0,0,"""""","""""",0.0,false,0,0,false,"""""",0,"""""","""""",false,false
